In [1]:
!pip install -U langgraph langsmith langchain_anthropic langchain-community tavily-python --quiet


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from typing import Annotated
from typing_extensions import TypedDict
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
##### CREATE STATE #####

from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

In [6]:
##### CREATE TOOLS #####

In [10]:
##### CREATE TOOLS: Tavily #####
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_search = TavilySearchResults(max_results=2)

In [11]:
##### COLLECT TOOLS #####
tools = [tavily_search]

In [13]:
##### BUILD LLM #####
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [17]:
##### BUILD GRAPH #####
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [18]:
##### TAKE A FEW STEPS #####

In [19]:
config = {"configurable": {"thread_id": "1"}}
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "I'm learning LangGraph. "
                    "Could you do some research on it for me?"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

I'm learning LangGraph. Could you do some research on it for me?
================================== Ai Message ==================================

[{'text': "Certainly! I'd be happy to research LangGraph for you. To get the most up-to-date and comprehensive information, I'll use the Tavily search engine to look this up. Let me do that for you now.", 'type': 'text'}, {'id': 'toolu_01PDCzuL1mpEnMCcxPt3BbpW', 'input': {'query': 'LangGraph programming framework'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_01PDCzuL1mpEnMCcxPt3BbpW)
 Call ID: toolu_01PDCzuL1mpEnMCcxPt3BbpW
  Args:
    query: LangGraph programming framework
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "LangGraph Tutorial: Building LLM Agents with LangChain's ... - Zep", "url": "https://www.getzep.com/a

In [20]:
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "Ya that's helpful. Maybe I'll "
                    "build an autonomous agent with it!"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Ya that's helpful. Maybe I'll build an autonomous agent with it!
================================== Ai Message ==================================

[{'text': "That's a great idea! Building an autonomous agent with LangGraph could be an excellent way to apply your learning and create something practical. Given your interest, let me provide some additional insights and suggestions for building an autonomous agent with LangGraph.", 'type': 'text'}, {'id': 'toolu_01FM8TVMgrwgx9QsV4vBAx9L', 'input': {'query': 'building autonomous agents with LangGraph examples'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_01FM8TVMgrwgx9QsV4vBAx9L)
 Call ID: toolu_01FM8TVMgrwgx9QsV4vBAx9L
  Args:
    query: building autonomous agents with LangGraph examples
================================= Tool Message =================================
Name: tavily_search_results_j

In [21]:
to_replay = None
for state in graph.get_state_history(config):
    print("Num Messages: ", len(state.values["messages"]), "Next: ", state.next)
    print("-" * 80)
    if len(state.values["messages"]) == 6:
        # We are somewhat arbitrarily selecting a specific state based on the number of chat messages in the state.
        to_replay = state

Num Messages:  8 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  7 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  6 Next:  ('tools',)
--------------------------------------------------------------------------------
Num Messages:  5 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  4 Next:  ('__start__',)
--------------------------------------------------------------------------------
Num Messages:  4 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  3 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  2 Next:  ('tools',)
--------------------------------------------------------------------------------
Num Messages:  1 Next:  ('chatbot',)
-----------------------------------------

In [22]:
print(to_replay.next)
print(to_replay.config)

('tools',)
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f006737-6381-6d67-8006-a821b5f84610'}}


In [23]:
# The `checkpoint_id` in the `to_replay.config` corresponds to a state we've persisted to our checkpointer.
for event in graph.stream(None, to_replay.config, stream_mode="values"):
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================

[{'text': "That's a great idea! Building an autonomous agent with LangGraph could be an excellent way to apply your learning and create something practical. Given your interest, let me provide some additional insights and suggestions for building an autonomous agent with LangGraph.", 'type': 'text'}, {'id': 'toolu_01FM8TVMgrwgx9QsV4vBAx9L', 'input': {'query': 'building autonomous agents with LangGraph examples'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_01FM8TVMgrwgx9QsV4vBAx9L)
 Call ID: toolu_01FM8TVMgrwgx9QsV4vBAx9L
  Args:
    query: building autonomous agents with LangGraph examples
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Build Smarter AI Agents in Minutes with LangGraph - Spheron's Blog", "url": "https://blog.spheron.network/build-smarter-ai-agents-